Ejercicio 1

In [124]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

# Definir la lista de hechizos
hechizos = ["Alohomora", "Expecto patronum", "Expelliarmus", "Lumos"]

# Cargar el modelo de spaCy
nlp = spacy.load("es_core_news_md")

# Aumentar el límite de longitud
nlp.max_length = 2000000  # Aumentamos el límite a 2 millones de caracteres

# Paso 1: Crear el componente de detección de hechizos
@spacy.language.Language.component("detectar_hechizos")
def detectar_hechizos(doc):
    # Crear el PhraseMatcher
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    
    # Convertir los hechizos en patrones
    patterns = [nlp.make_doc(hechizo) for hechizo in hechizos]
    
    # Añadir los patrones al matcher
    matcher.add("HECHIZOS", None, *patterns)
    
    # Buscar coincidencias
    matches = matcher(doc)
    
    # Crear las entidades para los hechizos encontrados
    hechizos_detectados = [Span(doc, start, end, label="HECHIZO") for _, start, end in matches]
    
    # Filtrar las entidades existentes para evitar solapamientos
    existing_ents = set(doc.ents)
    
    # Filtrar hechizos detectados para eliminar los que ya están en las entidades existentes
    hechizos_detectados_filtrados = [
        hechizo for hechizo in hechizos_detectados
        if not any(hechizo.start >= ent.start and hechizo.end <= ent.end for ent in existing_ents)
    ]
    
    # Añadir las entidades filtradas al doc
    doc.ents = list(doc.ents) + hechizos_detectados_filtrados
    
    return doc

# Añadir el componente al pipeline
nlp.add_pipe("detectar_hechizos", last=True)

# Cargar el texto de los libros desde los archivos
def cargar_texto_libro(ruta_archivo):
    """Carga el texto de un archivo, comenzando desde la línea 36 (índice 35)"""
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        lineas = f.readlines()
        texto = "".join(lineas[35:])  # índice 35 = línea 36 (empieza desde 0)
    return texto

# Ruta a los archivos de los libros
ruta_libro_2 = "J.K. Rowling - Harry Potter 2 - La Cámara Secreta.txt"
ruta_libro_7 = "J.K. Rowling - Harry Potter 7 - Las Reliquias de la Muerte.txt"

# Cargar los textos de los libros
texto_libro_2 = cargar_texto_libro(ruta_libro_2)
texto_libro_7 = cargar_texto_libro(ruta_libro_7)


In [115]:
print(texto_libro_2[:1000])


1
    
El peor cumpleaños


No era la primera vez que en el número 4 de Privet Drive estallaba una discusión durante el desayuno. A primera hora de la mañana, había despertado al señor Vernon Dursley un sonoro ulular procedente del dormitorio de su sobrino Harry.
¡Es la tercera vez esta semana! se quejó, sentado a la mesa. ¡Si no puedes dominar a esa lechuza, tendrá que irse a otra parte!
Harry intentó explicarse una vez más.
Es que se aburre. Está acostumbrada a dar una vuelta por ahí. Si pudiera dejarla salir aunque sólo fuera de noche...
¿Acaso tengo cara de idiota? gruñó tío Vernon, con restos de huevo frito en el poblado bigote. Ya sé lo que ocurriría si saliera la lechuza.
Cambió una mirada sombría con su esposa, Petunia.
Harry quería seguir discutiendo, pero un eructo estruendoso y prolongado de Dudley, el hijo de los Dursley, ahogó sus palabras.
¡Quiero más beicon!
Queda más en la sartén, ricura dijo tía Petunia, volviendo los ojos a su robusto hijo. Tenemos que ali

In [116]:
print(texto_libro_7[:1000])


      Los dos hombres aparecieron de la nada, a unos cuántos metros de un camino angosto e iluminado por la luna. Por un segundo se quedaron quietos, las varitas apuntando directamente al pecho de uno y otro; entonces, reconociéndose mutuamente, guardaron sus varitas dentro de sus capas y comenzaron a caminar a buen paso en la misma dirección. 

      -¿Noticias? - Preguntó el más alto de los dos.

      -Las mejores, - replicó Severus Snape.

      El camino estaba bordeado a la izquierda por plantas silvestres y zarzas a medio crecer, a la derecha por setos altos y perfectamente cortados. Las capas de los hombres se agitaban alrededor de sus tobillos mientras caminaban.

      -Pensé que íbamos tarde, - dijo Yaxley, sus toscos rasgos se escondían y mostraban mientras las ramas que sobresalían de los árboles bloqueaban la luz de luna. -Fue un poco más difícil de lo que pensé. Pero espero que vaya a quedar satisfecho. ¿Confías en que tu recibimiento será bueno?

      Snape asintió, p

In [117]:
# Procesar los textos con spaCy
doc2 = nlp(texto_libro_2)
doc7 = nlp(texto_libro_7)


In [119]:
# Recorremos los documentos y mostramos la cantidad de hechizos por tipo
for i, doc in [(2, doc2), (7, doc7)]:
    print(f"\n--> Libro {i}:")
    
    # Crear un diccionario para contar las entidades HECHIZO
    hechizos_por_tipo = {}
    
    # Contar las entidades de tipo "HECHIZO"
    for ent in doc.ents:
        if ent.label_ == "HECHIZO":
            hechizos_por_tipo[ent.text.lower()] = hechizos_por_tipo.get(ent.text.lower(), 0) + 1
    
    # Imprimir la cantidad total de hechizos
    total_hechizos = sum(hechizos_por_tipo.values())
    print(f"HECHIZO: {total_hechizos}")
    
    # Imprimir los hechizos y su frecuencia por tipo
    print("Tipo: hechizo")
    for hechizo, cantidad in sorted(hechizos_por_tipo.items()):
        print(f"    {hechizo}: {cantidad}")



--> Libro 2:
HECHIZO: 1
Tipo: hechizo
    lumos: 1

--> Libro 7:
HECHIZO: 7
Tipo: hechizo
    expelliarmus: 6
    lumos: 1


In [127]:
import spacy
from spacy import displacy
from IPython.display import display, HTML  # Importamos display y HTML manualmente


# Texto de ejemplo
texto = """Lanzó expeliarmus para defenderse. Como no dio resultado gritó "¡ALOHOMORA!, quería abriur la puerta y esconderse en la Sala de los Menesteres."""

# Procesar el texto con el modelo nlp
doc = nlp(texto)

# Fragmentar el texto en frases e imprimirlas
for i, sent in enumerate(doc.sents, 1):
    print(f"FRASE {i}")
    print(sent.text)
    print()  # Salto de línea para separar las frases

# Renderizar el texto con las dependencias (en lugar de entidades)
html = displacy.render(doc, style="ent", page=True, jupyter=False)

# Mostrar en Jupyter o fuera de él
display(HTML(html))


FRASE 1
Lanzó expeliarmus para defenderse.

FRASE 2
Como no dio resultado gritó "¡ALOHOMORA!, quería abriur la puerta y esconderse en la Sala de los Menesteres.



Ejercicio 2

> apartado 1)

In [71]:
#@Language.component

import spacy
from spacy.tokens import Span, Doc
from spacy.language import Language
import re

# Definimos las extensiones personalizadas para Span
Span.set_extension("numero", default=None, force=True)
Span.set_extension("titulo", default=None, force=True)
Span.set_extension("longitud", default=None, force=True)
Span.set_extension("contenido", default=None, force=True)

# Definimos la extensión para Doc
Doc.set_extension("capitulos", default=[], force=True)

# Componente que fragmenta los capítulos
@Language.component("fragmentador_capitulos")
def fragmentador_capitulos(doc):
    patron_capitulo = re.finditer(r'\n+(\d+)\s*\n+([^\n]+)', doc.text)
    capitulos = []  # Lista para almacenar los capítulos
    
    # # Procesar las coincidencias y crear los Spans
    # for match in patron_capitulo:
    #     numero = int(match.group(1))  # Número del capítulo
    #     titulo = match.group(2).strip()  # Título del capítulo
        
    #     # Obtener la posición de inicio y fin del capítulo
    #     start_char = match.start()
    #     end_char = match.end()
        
    #     # Crear Span para el capítulo
    #     span = doc.char_span(start_char, end_char, alignment_mode="expand")
        
    #     if span:
    #         span._.numero = numero
    #         span._.titulo = titulo
    #         span._.longitud = len(span)  # Número de tokens del capítulo
    #         capitulos.append(span)
    # CON LO COMENTADO NO NOS DIVIDÍA EL TEXTO DE LOS CAPÍTULOS (SOLO SE QUEDABA CON LOS TÍTULOS)


    # Obtener posiciones de inicio y fin de cada capítulo
    coincidencias = list(patron_capitulo)
    for i, match in enumerate(coincidencias):
        numero = match.group(1)
        titulo = match.group(2)

        start_char = match.start()
        # end_char = coincidencias[i + 1].start() if i + 1 < len(coincidencias) else len(texto)
        end_char = coincidencias[i + 1].start() if i + 1 < len(coincidencias) else len(doc.text)


        span = doc.char_span(start_char, end_char, alignment_mode="expand")

        if span:
            span._.numero = numero
            span._.titulo = titulo
            span._.longitud = len(span)

            # Extraer solo el contenido del capítulo (sin número ni título)
            cabecera_fin = match.end()
            cuerpo = doc.char_span(cabecera_fin, end_char, alignment_mode="expand")
            span._.contenido = cuerpo.text if cuerpo else ""

            capitulos.append(span)
    
    # Asignar la lista de capítulos al doc
    doc._.capitulos = capitulos
    return doc


> apartado 2)

LIBRO 1

In [72]:
nlp = spacy.load("es_core_news_md") 

# Añadir el componente al pipeline de spaCy
nlp.add_pipe("fragmentador_capitulos", last=True)

<function __main__.fragmentador_capitulos(doc)>

In [73]:
# Ruta al archivo (ajusta si está en otra carpeta)
ruta_archivo = "J.K. Rowling - Harry Potter 1 - La Piedra Filosofal.txt"

# Leer el archivo desde la línea 38 en adelante
with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto1 = "".join(lineas[37:])  # índice 37 = línea 38 (empieza desde 0)

print(texto1[:1000])  # Muestra los primeros 1000 caracteres para verificar




1
    
El niño que vivió


El señor y la señora Dursley, que vivían en el número 4 de Privet Drive, estaban orgullosos de decir que eran muy normales, afortunadamente. Eran las últimas personas que se esperaría encontrar relacionadas con algo extraño o misterioso, porque no estaban para tales tonterías.
El señor Dursley era el director de una empresa llamada Grunnings, que fabricaba taladros. Era un hombre corpulento y rollizo, casi sin cuello, aunque con un bigote inmenso. La señora Dursley era delgada, rubia y tenía un cuello casi el doble de largo de lo habitual, lo que le resultaba muy útil, ya que pasaba la mayor parte del tiempo estirándolo por encima de la valla de los jardines para espiar a sus vecinos. Los Dursley tenían un hijo pequeño llamado Dudley, y para ellos no había un niño mejor que él.
Los Dursley tenían todo lo que querían, pero también tenían un secreto, y su mayor temor era que lo descubriesen: no habrían soportado que se supiera lo de los Potter.
La señora Pot

In [74]:
doc1 = nlp(texto1)

In [75]:
# Verifica si se han detectado capítulos
if doc1._.capitulos:
    for capitulo in doc1._.capitulos:
        print(f"Capítulo {capitulo._.numero}: {capitulo._.titulo} - {capitulo._.longitud} tokens")
else:
    print("No se detectaron capítulos.")

Capítulo 1: El niño que vivió - 5609 tokens
Capítulo 2: El vidrio que se desvaneció - 4172 tokens
Capítulo 3: Las cartas de nadie - 4602 tokens
Capítulo 4: El guardián de las llaves - 4500 tokens
Capítulo 5: El callejón Diagon - 8144 tokens
Capítulo 6: El viaje desde el andén nueve y tres cuartos - 7719 tokens
Capítulo 7: El sombrero seleccionador - 5590 tokens
Capítulo 8: El profesor de pociones - 3672 tokens
Capítulo 9: El duelo a medianoche - 5935 tokens
Capítulo 10: Halloween - 4998 tokens
Capítulo 11: Quidditch - 4039 tokens
Capítulo 12: El espejo de Oesed - 6516 tokens
Capítulo 13: Nicolás Flamel - 3879 tokens
Capítulo 14: Norberto, el ridgeback noruego - 4126 tokens
Capítulo 15: El bosque prohibido - 6100 tokens
Capítulo 16: A través de la trampilla - 7705 tokens
Capítulo 17: El hombre con dos caras - 6788 tokens


In [76]:
import pandas as pd

df = pd.DataFrame([{
    "numero": cap._.numero,
    "titulo": cap._.titulo,
    "longitud_tokens": cap._.longitud,
    "texto": cap.text.strip()
} for cap in doc1._.capitulos])

df   


,numero,titulo,longitud_tokens,texto
0,1,El niño que vivió,5609,1\n \nEl niño que vivió\n\n\nEl señor y la ...
1,2,El vidrio que se desvaneció,4172,2\n\nEl vidrio que se desvaneció\n\n\nHabían p...
2,3,Las cartas de nadie,4602,3\n\nLas cartas de nadie\n\n\nLa fuga de la bo...
3,4,El guardián de las llaves,4500,4\n\nEl guardián de las llaves\n\n\nBUM. Llama...
4,5,El callejón Diagon,8144,5\n\nEl callejón Diagon\n\n\nHarry se despertó...
5,6,El viaje desde el andén nueve y tres cuartos,7719,6\n\nEl viaje desde el andén nueve y tres cuar...
6,7,El sombrero seleccionador,5590,7\n\nEl sombrero seleccionador\n\n\nLa puerta ...
7,8,El profesor de pociones,3672,"8\n\nEl profesor de pociones\n\n\nAllí, mira...."
8,9,El duelo a medianoche,5935,9\n\nEl duelo a medianoche\n\n\nHarry nunca ha...
9,10,Halloween,4998,10\n\nHalloween\n\n\nMalfoy no podía creer lo ...


LIBRO 2

In [77]:
# Ruta al archivo (ajusta si está en otra carpeta)
ruta_archivo = "J.K. Rowling - Harry Potter 2 - La Cámara Secreta.txt"

# Leer el archivo desde la línea 38 en adelante
with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto2 = "".join(lineas[35:])  # índice 35 = línea 36 (empieza desde 0)

print(texto2[:1000])  # Muestra los primeros 1000 caracteres para verificar


1
    
El peor cumpleaños


No era la primera vez que en el número 4 de Privet Drive estallaba una discusión durante el desayuno. A primera hora de la mañana, había despertado al señor Vernon Dursley un sonoro ulular procedente del dormitorio de su sobrino Harry.
¡Es la tercera vez esta semana! se quejó, sentado a la mesa. ¡Si no puedes dominar a esa lechuza, tendrá que irse a otra parte!
Harry intentó explicarse una vez más.
Es que se aburre. Está acostumbrada a dar una vuelta por ahí. Si pudiera dejarla salir aunque sólo fuera de noche...
¿Acaso tengo cara de idiota? gruñó tío Vernon, con restos de huevo frito en el poblado bigote. Ya sé lo que ocurriría si saliera la lechuza.
Cambió una mirada sombría con su esposa, Petunia.
Harry quería seguir discutiendo, pero un eructo estruendoso y prolongado de Dudley, el hijo de los Dursley, ahogó sus palabras.
¡Quiero más beicon!
Queda más en la sartén, ricura dijo tía Petunia, volviendo los ojos a su robusto hijo. Tenemos que ali

In [78]:
doc2 = nlp(texto2)

In [79]:
# Verifica si se han detectado capítulos
if doc2._.capitulos:
    for capitulo in doc2._.capitulos:
        print(f"Capítulo {capitulo._.numero}: {capitulo._.titulo} - {capitulo._.longitud} tokens")
else:
    print("No se detectaron capítulos.")

Capítulo 1: El peor cumpleaños - 3313 tokens
Capítulo 2: La advertencia de Dobby - 3840 tokens
Capítulo 3: La Madriguera - 5832 tokens
Capítulo 4: En Flourish y Blotts - 7522 tokens
Capítulo 5: El sauce boxeador - 6900 tokens
Capítulo 6: Gilderoy Lockhart - 5818 tokens
Capítulo 7: Los «sangre sucia» y una voz misteriosa - 5783 tokens
Capítulo 8: El cumpleaños de muerte - 5818 tokens
Capítulo 9: La inscripción en el muro - 6563 tokens
Capítulo 10: La «bludger» loca - 6693 tokens
Capítulo 11: El club de duelo - 7595 tokens
Capítulo 12: La poción «multijugos» - 6878 tokens
Capítulo 13: El diario secretísimo - 7003 tokens
Capítulo 14: Cornelius Fudge - 4895 tokens
Capítulo 15: Aragog - 5789 tokens
Capítulo 16: La Cámara de los Secretos - 7109 tokens
Capítulo 17: El heredero de Slytherin - 6736 tokens
Capítulo 18: La recompensa de Dobby - 4565 tokens


In [84]:
import pandas as pd

df = pd.DataFrame([{
    "numero": cap._.numero,
    "titulo": cap._.titulo,
    "longitud_tokens": cap._.longitud,
    "texto": cap.text.strip()
} for cap in doc2._.capitulos])

df   


,numero,titulo,longitud_tokens,texto
0,1,El peor cumpleaños,3313,1\n \nEl peor cumpleaños\n\n\nNo era la pri...
1,2,La advertencia de Dobby,3840,2\n\nLa advertencia de Dobby\n\n\nHarry no gri...
2,3,La Madriguera,5832,"3\n\nLa Madriguera\n\n\n¡Ron! exclamó Harry,..."
3,4,En Flourish y Blotts,7522,4\n\nEn Flourish y Blotts\n\n\nLa vida en La M...
4,5,El sauce boxeador,6900,5\n\nEl sauce boxeador\n\n\nEl final del veran...
5,6,Gilderoy Lockhart,5818,"6\n\nGilderoy Lockhart\n\n\nAl día siguiente, ..."
6,7,Los «sangre sucia» y una voz misteriosa,5783,7\n\nLos «sangre sucia» y una voz misteriosa\n...
7,8,El cumpleaños de muerte,5818,8\n\nEl cumpleaños de muerte\n\n\nLlegó octubr...
8,9,La inscripción en el muro,6563,9\n\nLa inscripción en el muro\n\n\n¿Qué pasa...
9,10,La «bludger» loca,6693,10\n\nLa «bludger» loca\n\n\nDespués del desas...


> apartado 3)

In [85]:
Doc.set_extension("titulo_libro", default=None, force=True)
# Títulos de los libros
titulos_libros = ["Piedra Filosofal", "Cámara Secreta"]

# Asignar a los documentos
doc1._.titulo_libro = titulos_libros[0]
doc2._.titulo_libro = titulos_libros[1]
print(doc1._.titulo_libro)  # Piedra Filosofal
print(doc2._.titulo_libro)  # Cámara Secreta


Piedra Filosofal
Cámara Secreta


> apartado 4)

para doc1

In [86]:
from spacy.tokens import Span

# Buscar todas las apariciones de "Hermione" en el doc
nuevas_entidades = []

for token in doc1:
    if token.text == "Hermione":
        # Crear un Span de un solo token
        span = Span(doc1, token.i, token.i + 1, label="PROTAGONISTA")
        nuevas_entidades.append(span)


In [88]:
from spacy.util import filter_spans

# Combinar y filtrar para eliminar solapamientos
todas_las_entidades = list(doc1.ents) + nuevas_entidades
doc1.ents = filter_spans(todas_las_entidades)


In [89]:
for ent in doc1.ents:
    if ent.label_ == "PROTAGONISTA":
        print(ent.text, ent.start, ent.end, ent.label_)


Hermione 45660 45661 PROTAGONISTA
Hermione 58823 58824 PROTAGONISTA


para doc2

In [92]:
from spacy.tokens import Span

# Buscar todas las apariciones de "Hermione" en el doc
nuevas_entidades = []

for token in doc2:
    if token.text == "Hermione":
        # Crear un Span de un solo token
        span = Span(doc2, token.i, token.i + 1, label="PROTAGONISTA")
        nuevas_entidades.append(span)

In [93]:
from spacy.util import filter_spans

# Combinar y filtrar para eliminar solapamientos
todas_las_entidades = list(doc2.ents) + nuevas_entidades
doc2.ents = filter_spans(todas_las_entidades)


In [94]:
for ent in doc2.ents:
    if ent.label_ == "PROTAGONISTA":
        print(ent.text, ent.start, ent.end, ent.label_)


> apartado 5

In [95]:
def contar_protagonistas(capitulo):
    return sum(1 for ent in capitulo.ents if ent.label_ == "PROTAGONISTA")

In [96]:
for cap in doc1._.capitulos:
    num_protagonistas = contar_protagonistas(cap)
    print(f"Capítulo {cap._.numero}: {cap._.titulo} - {num_protagonistas} menciones de PROTAGONISTA")


Capítulo 1: El niño que vivió - 0 menciones de PROTAGONISTA
Capítulo 2: El vidrio que se desvaneció - 0 menciones de PROTAGONISTA
Capítulo 3: Las cartas de nadie - 0 menciones de PROTAGONISTA
Capítulo 4: El guardián de las llaves - 0 menciones de PROTAGONISTA
Capítulo 5: El callejón Diagon - 0 menciones de PROTAGONISTA
Capítulo 6: El viaje desde el andén nueve y tres cuartos - 0 menciones de PROTAGONISTA
Capítulo 7: El sombrero seleccionador - 0 menciones de PROTAGONISTA
Capítulo 8: El profesor de pociones - 0 menciones de PROTAGONISTA
Capítulo 9: El duelo a medianoche - 1 menciones de PROTAGONISTA
Capítulo 10: Halloween - 0 menciones de PROTAGONISTA
Capítulo 11: Quidditch - 1 menciones de PROTAGONISTA
Capítulo 12: El espejo de Oesed - 0 menciones de PROTAGONISTA
Capítulo 13: Nicolás Flamel - 0 menciones de PROTAGONISTA
Capítulo 14: Norberto, el ridgeback noruego - 0 menciones de PROTAGONISTA
Capítulo 15: El bosque prohibido - 0 menciones de PROTAGONISTA
Capítulo 16: A través de la tra

In [97]:
for cap in doc2._.capitulos:
    num_protagonistas = contar_protagonistas(cap)
    print(f"Capítulo {cap._.numero}: {cap._.titulo} - {num_protagonistas} menciones de PROTAGONISTA")


Capítulo 1: El peor cumpleaños - 0 menciones de PROTAGONISTA
Capítulo 2: La advertencia de Dobby - 0 menciones de PROTAGONISTA
Capítulo 3: La Madriguera - 0 menciones de PROTAGONISTA
Capítulo 4: En Flourish y Blotts - 0 menciones de PROTAGONISTA
Capítulo 5: El sauce boxeador - 0 menciones de PROTAGONISTA
Capítulo 6: Gilderoy Lockhart - 0 menciones de PROTAGONISTA
Capítulo 7: Los «sangre sucia» y una voz misteriosa - 0 menciones de PROTAGONISTA
Capítulo 8: El cumpleaños de muerte - 0 menciones de PROTAGONISTA
Capítulo 9: La inscripción en el muro - 0 menciones de PROTAGONISTA
Capítulo 10: La «bludger» loca - 0 menciones de PROTAGONISTA
Capítulo 11: El club de duelo - 0 menciones de PROTAGONISTA
Capítulo 12: La poción «multijugos» - 0 menciones de PROTAGONISTA
Capítulo 13: El diario secretísimo - 0 menciones de PROTAGONISTA
Capítulo 14: Cornelius Fudge - 0 menciones de PROTAGONISTA
Capítulo 15: Aragog - 0 menciones de PROTAGONISTA
Capítulo 16: La Cámara de los Secretos - 0 menciones de P

> apartado 6

In [98]:
from collections import Counter

# Contador para verbos que siguen a un PROTAGONISTA
verbos_despues = Counter()

# Recorremos todos los capítulos del documento
for cap in doc1._.capitulos:
    for i, token in enumerate(cap):
        # Comprobar si el token actual es una entidad PROTAGONISTA
        if token.ent_type_ == "PROTAGONISTA":
            # Verificamos que hay un siguiente token
            if i + 1 < len(cap):
                siguiente = cap[i + 1] 
                if siguiente.pos_ == "VERB":
                    verbos_despues[siguiente.lemma_] += 1  # Usamos el lema para agrupar correctamente


In [99]:
print("Verbos más frecuentes después de PROTAGONISTA:")
for verbo, freq in verbos_despues.most_common(10):
    print(f"{verbo}: {freq}")

Verbos más frecuentes después de PROTAGONISTA:


In [100]:
from collections import Counter

# Contador para verbos que siguen a un PROTAGONISTA
verbos_despues = Counter()

# Recorremos todos los capítulos del documento
for cap in doc2._.capitulos:
    for i, token in enumerate(cap):
        # Comprobar si el token actual es una entidad PROTAGONISTA
        if token.ent_type_ == "PROTAGONISTA":
            # Verificamos que hay un siguiente token
            if i + 1 < len(cap):
                siguiente = cap[i + 1] 
                if siguiente.pos_ == "VERB":
                    verbos_despues[siguiente.lemma_] += 1  # Usamos el lema para agrupar correctamente

In [101]:
print("Verbos más frecuentes después de PROTAGONISTA:")
for verbo, freq in verbos_despues.most_common(10):
    print(f"{verbo}: {freq}")

Verbos más frecuentes después de PROTAGONISTA:
